In [1]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time

In [2]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [4]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(-1,28,28,1)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(-1,28,28,1)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

# Import network, layers and functions

In [5]:
from network import Sequential,layers
from layers import conv2d,max_pool,flatten,dense
import functions
import numpy as np

# Make Sequential Model

In [28]:
model=Sequential()

In [29]:
model.add(conv2d(input_shape=(28,28,1),num_kernels=16,kernel_size=5,activation=functions.relu))
model.add(max_pool())
model.add(conv2d(num_kernels=32,kernel_size=5,activation=functions.relu))
model.add(max_pool())
model.add(flatten())
model.add(dense(256,activation=functions.relu))
model.add(dense(64,activation=functions.relu))
model.add(dense(10,activation=functions.softmax))

# View Model Summary

In [30]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Name (type)                     Output Shape             Activation        Param #
input_layer (InputLayer)       (None, 28, 28, 1)          echo             0
__________________________________________________________________________________________
conv2d (conv2d)                (None, 28, 28, 16)         relu             416
__________________________________________________________________________________________
max_pool (max_pool)            (None, 14, 14, 16)         echo             0
__________________________________________________________________________________________
conv2d (conv2d)                (None, 14, 14, 32)         relu             12832
__________________________________________________________________________________________
max_pool (max_pool)            (None, 7, 7, 32)           echo             0
_______________________________________________________________________

# Compile model with optimizer, loss and lr

In [31]:
model.compile(optimizer=functions.adam,loss=functions.cross_entropy_with_logits,learning_rate=0.001)

# Train model

In [32]:
losses=[]
val_losses=[]
train_acc=[]
val_acc=[]

In [33]:
def run():
    st=0
    btsz=8
    cuts=(((60000*np.arange(11)/10))//btsz)*btsz
    while st<=60000:
        perc=st/600
        ck=np.random.randint(0,60000-btsz)
        inp,y_inp=batch_gen(ck,size=btsz)
        logits=model.fit(inp,y_inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc=100*(ans==cor).mean()
        cross_entropy_loss=model.loss(logits=logits,labels=y_inp).mean()
        losses.append(cross_entropy_loss)
        train_acc.append(acc)
        print("\rProgress: {:.2f} %\tAcc: {:.2f} %\tloss: {:.8f}".format(perc,acc,cross_entropy_loss),end='')
        if st in cuts:
            print("\nCalculating Validation acc...",end='')
            sz=4000
#             ck=0
            ck=np.random.randint(0,10000-sz)
            inp,y_inp=batch_gen(ck,size=sz,validate=True)
            logits=model.predict(inp)
            ans=logits.argmax(axis=1)
            cor=y_inp.argmax(axis=1)
            acc=100*(ans==cor).mean()
            cross_entropy_loss=model.loss(logits,labels=y_inp).mean()
            print("\rValidation Acc: {:.2f} %\t\tVal loss: {:.8f}".format(acc,cross_entropy_loss))
            val_losses.append(cross_entropy_loss)
            val_acc.append(acc)
            if acc>=96:
                model.learning_rate=0.00001
            elif acc>=93:
                model.learning_rate=0.0001
            elif acc>=90:
                model.learning_rate=0.001
        st+=btsz

In [ ]:
for i in range(10):
    run()

Progress: 0.00 %	Acc: 100.00 %	loss: 0.00018377
Validation Acc: 95.38 %		Val loss: 0.01519213
Progress: 10.00 %	Acc: 100.00 %	loss: 0.00011396
Validation Acc: 97.20 %		Val loss: 0.00869963
Progress: 20.00 %	Acc: 100.00 %	loss: 0.01608957
Validation Acc: 96.78 %		Val loss: 0.01035080
Progress: 30.00 %	Acc: 100.00 %	loss: 0.00009648
Validation Acc: 96.45 %		Val loss: 0.01130031
Progress: 40.00 %	Acc: 100.00 %	loss: 0.00020198
Validation Acc: 98.88 %		Val loss: 0.00387886
Progress: 50.00 %	Acc: 100.00 %	loss: 0.00430515
Validation Acc: 96.58 %		Val loss: 0.01080264
Progress: 60.00 %	Acc: 100.00 %	loss: 0.00010976
Validation Acc: 96.90 %		Val loss: 0.00965760
Progress: 70.00 %	Acc: 100.00 %	loss: 0.00137612
Validation Acc: 97.80 %		Val loss: 0.00720352
Progress: 80.00 %	Acc: 100.00 %	loss: 0.00016424
Validation Acc: 98.92 %		Val loss: 0.00378992
Progress: 90.00 %	Acc: 87.50 %	loss: 0.143654791
Validation Acc: 99.05 %		Val loss: 0.00332965
Progress: 100.00 %	Acc: 87.50 %	loss: 0.01912425
Va

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(losses,color='#ff7f0e')
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(val_losses,color='#ff7f0e')
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(val_acc,color='#ff7f0e')
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(train_acc,color='#ff7f0e')
plt.show()

In [ ]:
ck=np.random.randint(0,60000-100)
inp,y_inp=batch_gen(ck,size=1)
logits=model.predict(inp)
ans=logits.argmax(axis=1)
cor=y_inp.argmax(axis=1)
print("Prediction:",ans,"\nCorrect:   ",cor)
plt.imshow(inp[0].reshape(28,28), cmap='Greys')

In [ ]:
fig, ax = plt.subplots(nrows=4,ncols=4,figsize=[10,10])
fig.patch.set_facecolor('white')
pme=model.sequence[2].z_out.transpose(3,0,1,2)
for i,axi in enumerate(ax.flat):
    axi.imshow(pme[i].reshape(pme.shape[-2:]), cmap='Greys')